<a href="https://colab.research.google.com/github/dasmiq/CS6120-HW2/blob/master/LanguageModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Your task is to train *character-level* language models. 
You will train unigram, bigram, and trigram character-level models on a collection of books from Project Gutenberg. You will then use these trained English language models to distinguish English documents from Brazilian Portuguese documents in the test set.

In [1]:
from nltk.util import ngrams, flatten
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from string import punctuation
import httpimport
import re
import pandas as pd
import numpy as np

with httpimport.remote_repo(['lm_helper'], 'https://raw.githubusercontent.com/jasoriya/CS6120-PS2-support/master/utils/'):
  from lm_helper import get_train_data, get_test_data

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Sam\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Sam\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package mac_morpho to
[nltk_data]     C:\Users\Sam\AppData\Roaming\nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!


This code loads the training and test data. Each dataset is a list of books. Each book contains a list of sentences, and each sentence contains a list of words. For building a character language model, you should join the words of a sentence together with a space character.

In [2]:
# get the train and test data
train = get_train_data()
test, test_files = get_test_data()

# split the training data 
train, train_left_out = train_test_split(train, test_size=.2)

## 1.1
Collect statistics on the unigram, bigram, and trigram character counts.

If your machine takes a long time to perform this computation, you may save these counts to files in your github repository and load them on request. This is not necessary, however.

In [3]:
uniCount = FreqDist()
biCount = FreqDist()
triCount = FreqDist()
uniGrams = []
biGrams = []
triGrams = []

for book in train:
    for sentence in book:
        s = ' '.join(sentence).lower()
        # # get rid of numbers
        # s = re.sub("\d+", "", s).lower()
        # # remove punctuation
        # s = s.translate(str.maketrans('', '', punctuation))
        # if s and s[-1] == ' ':
        #     s = s[:-1] # gets rid of any trailling spaces
        uni = ngrams(s, 1)
        bi = ngrams(s, 2)
        tri = ngrams(s, 3) 

        # update counters
        un = [u for u in uni]
        bn = [b for b in bi]
        tn = [t for t in tri]
        uniCount.update([''.join(u) for u in un])
        biCount.update([''.join(b) for b in bn])
        triCount.update([''.join(t) for t in tn])
        uniGrams.append(un)
        biGrams.append(bn)
        triGrams.append(tn)




In [4]:
print(uniCount.most_common(10))
print(biCount.most_common(10))
print(triCount.most_common(10))

[(' ', 2230425), ('e', 981459), ('t', 728992), ('a', 643403), ('o', 593513), ('h', 579387), ('n', 539058), ('i', 503494), ('s', 489093), ('r', 436650)]
[('e ', 377235), (' t', 308978), ('th', 298761), ('he', 258362), ('d ', 236991), (' a', 211213), ('t ', 189643), ('s ', 189439), (' ,', 171789), (', ', 167231)]
[(' th', 231760), ('the', 185521), (' , ', 167023), ('he ', 150205), ('nd ', 104299), ('and', 99411), (' an', 97037), (' of', 68307), ('of ', 65672), ('ed ', 57146)]


## 1.2
Calculate the perplexity for each document in the test set using the linear interpolation smoothing method. For determining λs for linear interpolation, you can divide the training data into a new training set (80%) and a held-out set (20%), then using grid search method:
Choose ~10 values of λ to test using grid search on held-out data.

Some documents in the test set are in Brazilian Portuguese. Identify them as follows: 
  - Sort by perplexity and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names (from `test_files`) and perplexities of the documents above the threshold

    ```
        file name, score
        file name, score
        . . .
        file name, score
    ```

  - Copy this list of filenames and manually annotate them as being correctly or incorrectly labeled as Portuguese.




In [5]:
def perplexity(lambda_1, lambda_2, lambda_3, tri_count, bi_count, uni_count, document):
    tokenized_document = []
    for sentence in document:
        s = ' '.join(sentence).lower()
        tri = ngrams(s, 3)
        tokenized_document.append([''.join(token) for token in tri])

    tokenized_document = flatten(tokenized_document)
    probs = []
    for token in tokenized_document: # calc the probability
        p3 = (lambda_3
         * (tri_count[token]
         /bi_count.get(token[:-1], 1))) + (lambda_2 * (
             bi_count[token[:-1]]
             /uni_count.get(token[:-2], 1))) + (lambda_3 * 
             (uni_count[token[:-2]]
             /sum(uni_count.values())))
        
        if p3 == 0:
            p3 = 1/sum(uni_count.values())
        
        probs.append(p3)
    
    l_probs = sum(np.log10(probs))
    return 2**(-l_probs/len(tokenized_document))

    


lambdas = [
    [.35, .24, .41],
    [.225, .125, .750],
    [.56, .24, .1],
    [.3, .5, .2],
    [.35, .20, .45],
    [.40, .40, .2],
    [.12, .24, .64],
    [.123, .522, .355],
    [.66, .155, .185],
    [.250, .60, .15]
]

best_lambda = {}
for ls in lambdas:
    perlex_list = []
    for document in train_left_out:
        perlex_list.append(perplexity(ls[0], ls[1], ls[2], triCount, biCount, uniCount, document))
    
    best_lambda[sum(perlex_list)/len(perlex_list)] = ls


In [17]:
lambdas_to_use = best_lambda[min(best_lambda.keys())]

brazillian_docs = []

for document, file_name in zip(test, test_files):
    perplex = perplexity(lambdas_to_use[0], lambdas_to_use[1], lambdas_to_use[2], triCount, biCount, uniCount, document)
    threshold = 1.75
    if perplex > threshold:
        brazillian_docs.append([file_name, perplex, document])


In [18]:
for d  in [[_[0], _[1]] for _ in brazillian_docs]:
    print(d)



['ag94ma03.txt', 2.131517746123798]
['br94jl01.txt', 2.102406366439263]
['ag94ou04.txt', 2.106702704371739]
['ag94mr1.txt', 2.102899858413237]
['br94fe1.txt', 2.1309273260011725]
['br94ju01.txt', 2.101198034130712]
['br94ma01.txt', 2.098401522494839]
['br94ab02.txt', 2.0930243421247345]
['ag94ag02.txt', 2.1089128763593705]
['ag94ab12.txt', 2.1240380191712336]
['br94ag01.txt', 2.113743064919677]
['ag94no01.txt', 2.108940515898756]
['ag94fe1.txt', 2.080846596592233]
['ag94se06.txt', 2.12907451126396]
['ag94de06.txt', 2.105830886340917]
['ag94ju07.txt', 2.125475699045007]
['ag94jl12.txt', 2.1159451021069584]
['br94de01.txt', 2.102674427746029]
['ag94ja11.txt', 2.094127710230229]
['br94ja04.txt', 2.114897223736423]


'ag94ab12.txt', correct

'ag94mr1.txt', correct

'br94ma01.txt', correct

'ag94se06.txt', correct

'ag94ja11.txt', correct

'ag94ju07.txt', correct

'br94fe1.txt', correct

'ag94ag02.txt', correct

'ag94de06.txt', correct

'ag94ma03.txt', correct

'ag94fe1.txt', correct

'ag94jl12.txt', correct

'ag94no01.txt', correct

'br94ab02.txt', correct

'br94de01.txt', correct

'br94ag01.txt', correct

'ag94ou04.txt', correct

'br94jl01.txt', correct

'br94ja04.txt', correct

'br94ju01.txt', correct


# 1.3
Build a trigram language model with add-λ smoothing (use λ = 0.1).

Sort the test documents by perplexity and perform a check for Brazilian Portuguese documents as above:

  - Observe the perplexity scores and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names and perplexities of the documents above the threshold

  ```
      file name, score
      file name, score
      . . .
      file name, score
  ```

  - Copy this list of filenames and manually annotate them for correctness.

In [8]:
def perplexity_add_lambda(lambda_v, tri_count, bi_count, uni_count, document):
    tokenized_document = []
    for sentence in document:
        s = ' '.join(sentence).lower()
        tri = ngrams(s, 3)
        tokenized_document.append([''.join(token) for token in tri])

    tokenized_document = flatten(tokenized_document)
    probs = []
    for token in tokenized_document: # calc the probability
        p3 = (tri_count[token]/bi_count.get(token[:-1], 1)) + lambda_v
        
        probs.append(p3)
    
    l_probs = sum(np.log10(probs))
    return 2**(-l_probs/len(tokenized_document))

In [9]:
brazillian_docs_add_lambda = []

for document, file_name in zip(test, test_files):
    perplex = perplexity_add_lambda(.1, triCount, biCount, uniCount, document)
    threshold = 1.75
    if perplex > threshold:
        brazillian_docs_add_lambda.append([file_name, perplex, document])

In [10]:
for d  in [[_[0], _[1]] for _ in brazillian_docs_add_lambda]:
    print(d)


In [13]:
brazillian_docs_add_lambda_low = []

for document, file_name in zip(test, test_files):
    perplex = perplexity_add_lambda(.1, triCount, biCount, uniCount, document)
    threshold = 1.65
    if perplex > threshold:
        brazillian_docs_add_lambda_low.append([file_name, perplex, document])

In [14]:
for d  in [[_[0], _[1]] for _ in brazillian_docs_add_lambda_low]:
    print(d)


['ag94ma03.txt', 1.667372849268755]
['br94jl01.txt', 1.6604762968418614]
['ag94ou04.txt', 1.6662110547901836]
['ag94mr1.txt', 1.6629577287153132]
['br94fe1.txt', 1.6631193298834606]
['br94ju01.txt', 1.6611312547962334]
['br94ma01.txt', 1.6663247480456658]
['br94ab02.txt', 1.6617274730284959]
['ag94ag02.txt', 1.6635803647612344]
['ag94ab12.txt', 1.6638829627942946]
['br94ag01.txt', 1.6626830130604813]
['ag94no01.txt', 1.661853422442727]
['ag94fe1.txt', 1.6588192089402212]
['ag94se06.txt', 1.6663991325896095]
['ag94de06.txt', 1.6620537942864082]
['ag94ju07.txt', 1.6671296104540596]
['ag94jl12.txt', 1.6628946488958636]
['br94de01.txt', 1.6619672646456796]
['ag94ja11.txt', 1.6639252414763903]
['br94ja04.txt', 1.6601570368734788]


## 1.4
Based on your observation from above questions, compare linear interpolation and add-λ smoothing by listing out their pros and cons.

When comparing the two models made in this notebook it is important to see that with the threshold selected (1.75) the simple add k or add λ smoothing was not able to select any documents as being not-english. But taking a closer look at the data we can see that with a slight modification with a threshold of 1.65 we do get the documents back, but the problem is we still recieve documents which are english which have been classifed as non-english. we can move on to the pros and cons

Linear Interpolation

    Pros:
        Robust, linear Interpolation gives a more realistic probability of a character appearing because of the backoff model. The model allows for less data to be used since we are augmenting our dataset with the 
        bigrams and unigramas. The model seemingly seperates the documents slightly better with the difference between the highest scoring english document and the lowest scoring non english document being rather far 
        apart in comparission to the add k model.

    Cons:
        Requires more computation, since we have to calculate the bigram, and unigrams as well as the trigrams the model requires more computation.

Add K/λ smoothing
    
    Pros:
        Simple, doesn't require much preprocessing since we just need the trigrams this allows for rapid development of the model.
    
    Cons:
        the more robust you want the model you need a much larger dataset compred to the Linear Interpolation model.
        
